In [4]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.providers.basic_provider import BasicSimulator

class Simulator:
    def __init__(self, N, H):
        self.N = N
        self.H = H
    
    def classical(self):
        """Simulate the system classically."""
        best_energy = float('inf')
        best_config = None
        
        # Brute-force over all spin configurations
        for config in range(1 << self.N):
            spins = [1 if (config & (1 << i)) else -1 for i in range(self.N)]
            energy = self.H(spins)
            if energy < best_energy:
                best_energy = energy
                best_config = spins
        
        return best_config, best_energy
    
    def quantum(self, t=1.0, shots=1024):
        """Simulate the system using NumPy."""
        # Number of basis states (2^N)
        num_states = 2**self.N
        
        # Initial state: equal superposition of all computational basis states
        state = np.ones(num_states, dtype=complex) / np.sqrt(num_states)
        
        # Define the Hamiltonian matrix
        H_matrix = np.zeros((num_states, num_states), dtype=complex)
        for config in range(num_states):
            spins = [1 if (config & (1 << i)) else -1 for i in range(self.N)]
            energy = self.H(spins)
            H_matrix[config, config] = energy  # Diagonal Hamiltonian (classical Ising model)

        # Time evolution: U = e^(-iHt)
        U = np.linalg.matrix_power(np.exp(-1j * H_matrix * t), 1)

        # Apply time evolution to the state
        evolved_state = U @ state

        # Measure in the computational basis
        probabilities = np.abs(evolved_state)**2

        # Normalize the probabilities to avoid numerical errors
        probabilities /= np.sum(probabilities)

        # Sample outcomes based on the probabilities
        outcomes = np.random.choice(num_states, size=shots, p=probabilities)

        # Convert outcomes to bitstrings
        def config_to_bitstring(config):
            return bin(config)[2:].zfill(self.N)
        
        counts = {config_to_bitstring(outcome): 0 for outcome in outcomes}
        for outcome in outcomes:
            bitstring = config_to_bitstring(outcome)
            counts[bitstring] += 1

        # Map bitstrings to spin configurations
        def bitstring_to_spins(bitstring):
            return [1 if bit == '0' else -1 for bit in bitstring]

        # Compute energies for each observed state
        energies = {
            bitstring: self.H(bitstring_to_spins(bitstring))
            for bitstring in counts
        }

        # Find the configuration with the minimum energy
        best_bitstring = min(energies, key=energies.get)
        best_energy = energies[best_bitstring]
        best_spins = bitstring_to_spins(best_bitstring)
        
        return best_spins, best_energy
    
    def qiskit(self, t=1.0, shots=1024):
        """Simulate the system using quantum circuits."""
        # Define the time-evolution operator for H
        qr = QuantumRegister(self.N)
        cr = ClassicalRegister(self.N)
        qc = QuantumCircuit(qr, cr)

        # Start in the equal superposition state
        qc.h(qr)
        
        # Add Ising Hamiltonian time evolution
        for i in range(self.N):
            for j in range(i + 1, self.N):
                # Example Ising coupling term H = Z_i Z_j
                qc.rzz(2 * t, qr[i], qr[j])

        # Measure in computational basis
        qc.measure(qr, cr)
        
        simulator = BasicSimulator()
        compiled_circuit = transpile(qc, simulator)
        job = simulator.run(compiled_circuit, shots=shots)
        result = job.result()
        counts = result.get_counts()
        
        # Map bitstrings to spin configurations
        def bitstring_to_spins(bitstring):
            return [1 if bit == '0' else -1 for bit in bitstring]
        
        energies = {
            bitstring: self.H(bitstring_to_spins(bitstring))
            for bitstring in counts
        }
        best_bitstring = min(energies, key=energies.get)
        best_energy = energies[best_bitstring]
        best_spins = bitstring_to_spins(best_bitstring)
        
        return best_spins, best_energy
    
    def compute_eigenvalues(self):
        """Compute all possible eigenvalues of the Hamiltonian."""
        eigenvalues = set()
        for config in range(1 << self.N):
            spins = [1 if (config & (1 << i)) else -1 for i in range(self.N)]
            eigenvalue = self.H(spins)
            eigenvalues.add(eigenvalue)
        return sorted(eigenvalues)

    def suggest_t(self):
        """Suggest an optimal time t based on the energy gap."""
        eigenvalues = self.compute_eigenvalues()
        if len(eigenvalues) < 2:
            raise ValueError("Insufficient eigenvalues to compute energy gap.")

        # Compute the smallest non-zero energy gap
        gaps = [eigenvalues[i + 1] - eigenvalues[i] for i in range(len(eigenvalues) - 1)]
        delta_E = min(gaps)
        return 1 / delta_E  # Suggest t proportional to the inverse gap
    

In [5]:
class NumberPartitioning:
    def __init__(self, numbers):
        self.numbers = numbers
    
    def hamiltonian(self):
        """Define the Ising Hamiltonian for the problem."""
        # H = A * (sum(n_i * s_i))^2
        return lambda spins: (np.sum(np.array(self.numbers) * np.array(spins))) ** 2

# Example Input
numbers = [3, 1, 4, 2, 2]  # Example set of numbers
np_problem = NumberPartitioning(numbers)

sim = Simulator(len(numbers), np_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)


Classical Result: ([1, 1, -1, 1, -1], 0)
Suggested optimal t: 0.25
Quantum Result: ([1, 1, -1, 1, -1], 0)
Qiskit Result: ([1, 1, -1, 1, -1], 0)


In [7]:
class GraphPartitioning:
    def __init__(self, edges):
        self.edges = edges  # List of tuples representing edges [(u, v), ...]

    def hamiltonian(self):
        """Define the Ising Hamiltonian for graph partitioning."""
        def hamiltonian(spins):
            # First term: size constraint
            term1 = (np.sum(spins)) ** 2
            
            # Second term: edge penalty
            term2 = sum((1 - spins[u] * spins[v]) / 2 for u, v in self.edges)
            
            # Combine terms
            A, B = 1.0, 1.0  # Weights for terms
            return A * term1 + B * term2
        return hamiltonian


# Example Input
edges = [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2)]  # Example graph (a square with a diagonal)
num_vertices = 4
gp_problem = GraphPartitioning(edges)

sim = Simulator(num_vertices, gp_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
eigenvalues = sim.compute_eigenvalues()
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)


Classical Result: ([1, 1, -1, -1], 3.0)
[3.0, 4.0, 6.0, 7.0, 16.0]
Suggested optimal t: 1.0
Quantum Result: ([-1, -1, 1, 1], 3.0)
Qiskit Result: ([1, 1, -1, -1], 3.0)


In [8]:
class Cliques:
    def __init__(self, edges, k):
        self.edges = edges  # List of tuples representing edges [(u, v), ...]
        self.k = k  # Desired clique size

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the clique problem."""
        def hamiltonian(spins):
            # First term: enforce clique size
            term1 = (self.k - np.sum(spins)) ** 2
            
            # Second term: enforce clique completeness
            term2 = (self.k * (self.k - 1)) / 2 - sum(
                spins[u] * spins[v] for u, v in self.edges
            )
            
            # Combine terms
            A, B = 1.0, 1.0  # Weights for terms
            return A * term1 + B * term2
        
        return hamiltonian
    
# Example Input
edges = [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2)]  # Example graph
num_vertices = 4
k = 3  # Desired clique size
cliques_problem = Cliques(edges, k)

sim = Simulator(num_vertices, cliques_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([1, 1, 1, 1], -1.0)
Suggested optimal t: 0.5
Quantum Result: ([1, 1, 1, 1], -1.0)
Qiskit Result: ([1, 1, 1, 1], -1.0)


In [28]:
class LogSpinReduction:
    def __init__(self, num_states):
        self.num_states = num_states  # Number of possible states (N)
        self.num_spins = int(np.ceil(np.log2(num_states)))  # Logarithmic reduction
        self.H = None

    def hamiltonian(self):
        """Define a Hamiltonian that enforces constraints using log spins."""
        def hamiltonian(spins):
            # Convert spins to binary variables
            x = [(spin + 1) // 2 for spin in spins]  # Map spins {-1, 1} -> {0, 1}

            # Compute the integer value represented by the spins
            value = sum(2 ** i * x[i] for i in range(len(x)))

            # Constraint: value should be in the range [1, num_states]
            if value < 1 or value > self.num_states:
                return 1e6  # Large penalty for invalid states

            # Example Hamiltonian: penalize deviation from a target value
            target = self.num_states // 2  # Target value for demonstration
            return (value - target) ** 2

        return hamiltonian

num_states = 10  # Number of possible states (N)
log_spin_problem = LogSpinReduction(num_states)

sim = Simulator(num_states, log_spin_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([1, -1, 1, -1, -1, -1, -1, -1, -1, -1], 0)
Suggested optimal t: 1.0
Quantum Result: ([1, -1, 1, -1, -1, -1, -1, -1, -1, -1], 0)
Qiskit Result: ([1, -1, 1, -1, -1, -1, -1, -1, -1, -1], 0)


In [31]:
class BinaryIntegerLinearProgramming:
    def __init__(self, objective_coeffs, constraint_matrix, constraint_bounds):
        """
        Initialize the BILP problem.
        :param objective_coeffs: Coefficients for the objective function (list of c_i).
        :param constraint_matrix: Matrix of coefficients for constraints (list of lists a_{ij}).
        :param constraint_bounds: Bounds for the constraints (list of b_j).
        """
        self.objective_coeffs = objective_coeffs
        self.constraint_matrix = np.array(constraint_matrix)
        self.constraint_bounds = np.array(constraint_bounds)
        self.num_variables = len(objective_coeffs)
        self.H = None

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the BILP problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_i = (s_i + 1) / 2)
            x = (np.array(spins) + 1) / 2

            # Objective term
            H_objective = np.dot(self.objective_coeffs, x)

            # Penalty terms for constraints
            H_constraints = 0
            for j in range(len(self.constraint_bounds)):
                violation = max(0, np.dot(self.constraint_matrix[j], x) - self.constraint_bounds[j])
                H_constraints += violation ** 2

            # Combine terms
            A, B = 1.0, 1.0  # Weights for objective and constraints
            return A * H_objective + B * H_constraints

        return hamiltonian

# Example Input
objective_coeffs = [3, 5, 2, 7]  # Objective: Minimize 3x1 + 5x2 + 2x3 + 7x4
constraint_matrix = [
    [1, 0, 1, 1],  # x1 + x3 + x4 <= 2
    [0, 1, 1, 0],  # x2 + x3 <= 1
]
constraint_bounds = [2, 1]

bilp_problem = BinaryIntegerLinearProgramming(objective_coeffs, constraint_matrix, constraint_bounds)

sim = Simulator(len(objective_coeffs), bilp_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, -1, -1, -1], 0.0)
Suggested optimal t: 1.0
Quantum Result: ([-1, -1, -1, -1], 0.0)
Qiskit Result: ([-1, -1, -1, -1], 0.0)


In [32]:
class ExactCover:
    def __init__(self, universe, subsets):
        """
        Initialize the Exact Cover problem.
        :param universe: The universe of elements (list of integers).
        :param subsets: A collection of subsets (list of lists).
        """
        self.universe = universe
        self.subsets = subsets

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Exact Cover problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_i = (s_i + 1) / 2)
            x = (np.array(spins) + 1) / 2

            # Constraint term: Ensures each element in U is covered exactly once
            H_constraint = 0
            for elem in self.universe:
                cover_count = sum(x[i] for i, subset in enumerate(self.subsets) if elem in subset)
                H_constraint += (1 - cover_count) ** 2

            # Objective term: Minimize the number of chosen subsets
            H_objective = np.sum(x)

            # Combine terms
            A, B = 10.0, 1.0  # Weights for terms
            return A * H_constraint + B * H_objective

        return hamiltonian
        
universe = [1, 2, 3, 4]  # Universe U = {1, 2, 3, 4}
subsets = [[1, 2], [2, 3], [3, 4], [1, 4]]  # Subsets S_1 = {1, 2}, etc.

exact_cover_problem = ExactCover(universe, subsets)

sim = Simulator(len(subsets), exact_cover_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([1, -1, 1, -1], 2.0)
Suggested optimal t: 1.0
Quantum Result: ([1, -1, 1, -1], 2.0)
Qiskit Result: ([-1, 1, -1, 1], 2.0)


In [33]:
class SetPacking:
    def __init__(self, subsets):
        """
        Initialize the Set Packing problem.
        :param subsets: A collection of subsets (list of lists).
        """
        self.subsets = subsets
        self.universe = set(item for subset in subsets for item in subset)

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Set Packing problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_i = (s_i + 1) / 2)
            x = (np.array(spins) + 1) / 2

            # Constraint term: Ensures subsets are disjoint
            H_constraint = 0
            for elem in self.universe:
                cover_count = sum(x[i] for i, subset in enumerate(self.subsets) if elem in subset)
                H_constraint += (cover_count - 1) ** 2

            # Objective term: Maximize the number of selected subsets
            H_objective = -np.sum(x)

            # Combine terms
            A, B = 10.0, 1.0  # Weights for terms
            return A * H_constraint + B * H_objective

        return hamiltonian
    
        
subsets = [[1, 2], [2, 3], [3, 4], [1, 4]]  # Subsets S_1 = {1, 2}, etc.
set_packing_problem = SetPacking(subsets)

sim = Simulator(len(subsets), set_packing_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([1, -1, 1, -1], -2.0)
Suggested optimal t: 1.0
Quantum Result: ([1, -1, 1, -1], -2.0)
Qiskit Result: ([1, -1, 1, -1], -2.0)


In [34]:
class VertexCover:
    def __init__(self, vertices, edges):
        """
        Initialize the Vertex Cover problem.
        :param vertices: List of vertices (e.g., [0, 1, 2, 3]).
        :param edges: List of edges (e.g., [(0, 1), (1, 2), (2, 3), (3, 0)]).
        """
        self.vertices = vertices
        self.edges = edges

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Vertex Cover problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_i = (s_i + 1) / 2)
            x = (np.array(spins) + 1) / 2

            # Objective term: Minimize the number of selected vertices
            H_objective = np.sum(x)

            # Constraint term: Ensure all edges are covered
            H_constraint = 0
            for u, v in self.edges:
                H_constraint += 1 - x[u] - x[v] + x[u] * x[v]

            # Combine terms
            A, B = 10.0, 1.0  # Weights for terms
            return A * H_constraint + B * H_objective

        return hamiltonian
            
vertices = [0, 1, 2, 3]  # Vertices of the graph
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]  # Edges of the graph

vertex_cover_problem = VertexCover(vertices, edges)

sim = Simulator(len(vertices), set_packing_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([1, -1, 1, -1], -2.0)
Suggested optimal t: 1.0
Quantum Result: ([-1, 1, -1, 1], -2.0)
Qiskit Result: ([-1, 1, -1, 1], -2.0)


In [36]:
class Satisfiability:
    def __init__(self, clauses):
        """
        Initialize the SAT problem.
        :param num_variables: Number of boolean variables (e.g., 3 for x1, x2, x3).
        :param clauses: List of clauses, where each clause is a list of literals.
                        Positive literals are represented as integers (e.g., 1 for x1),
                        and negative literals are represented as negatives (e.g., -2 for ~x2).
        """
        self.clauses = clauses

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the SAT problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_i = (s_i + 1) / 2)
            x = (np.array(spins) + 1) / 2

            # Clause satisfaction
            H = 0
            for clause in self.clauses:
                # Compute the penalty for the clause
                clause_penalty = 1
                for literal in clause:
                    if literal > 0:
                        clause_penalty *= (1 - x[literal - 1])  # x_i for positive literals
                    else:
                        clause_penalty *= x[abs(literal) - 1]  # ~x_i for negative literals
                H += clause_penalty

            return H

        return hamiltonian

# Example Input
num_variables = 3  # Variables: x1, x2, x3
clauses = [[1, -2], [-1, 3], [2, -3]]  # Clauses: (x1 OR ~x2), (~x1 OR x3), (x2 OR ~x3)

satisfiability_problem = Satisfiability(clauses)

sim = Simulator(num_variables, satisfiability_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, -1, -1], 0.0)
Suggested optimal t: 1.0
Quantum Result: ([1, 1, 1], 0.0)
Qiskit Result: ([1, 1, 1], 0.0)


In [37]:
class MinimalMaximalMatching:
    def __init__(self, vertices, edges):
        """
        Initialize the Minimal Maximal Matching problem.
        :param vertices: List of vertices (e.g., [0, 1, 2, 3]).
        :param edges: List of edges (e.g., [(0, 1), (1, 2), (2, 3), (3, 0)]).
        """
        self.vertices = vertices
        self.edges = edges

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Minimal Maximal Matching problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_e = (s_e + 1) / 2)
            x = (np.array(spins) + 1) / 2

            # Matching constraint: No two edges share a vertex
            H_matching = 0
            for v in self.vertices:
                incident_edges = [i for i, e in enumerate(self.edges) if v in e]
                H_matching += sum(x[i] for i in incident_edges) ** 2

            # Maximality constraint: Ensure each edge not in M is covered
            H_maximality = 0
            for e in self.edges:
                edge_covered = 1
                for v in e:
                    edge_covered *= (1 - sum(x[i] for i, e_prime in enumerate(self.edges) if v in e_prime))
                H_maximality += edge_covered

            # Objective term: Minimize the number of edges in the matching
            H_objective = np.sum(x)

            # Combine terms
            A, B, C = 10.0, 10.0, 1.0  # Weights for constraints and objective
            return A * H_matching + B * H_maximality + C * H_objective

        return hamiltonian

vertices = [0, 1, 2, 3]  # Vertices of the graph
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]  # Edges of the graph

minimal_matching_problem = MinimalMaximalMatching(vertices, edges)

sim = Simulator(len(edges), minimal_matching_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([1, -1, -1, -1], 31.0)
Suggested optimal t: 0.5
Quantum Result: ([1, -1, -1, -1], 31.0)
Qiskit Result: ([1, -1, -1, -1], 31.0)


In [38]:
# Let's come back and check the Hamiltonian here.
class SetCover:
    def __init__(self, universe, subsets):
        """
        Initialize the Set Cover problem.
        :param universe: The universe of elements (list of integers).
        :param subsets: A collection of subsets (list of lists).
        """
        self.universe = universe
        self.subsets = subsets

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Set Cover problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_i = (s_i + 1) / 2)
            x = (np.array(spins) + 1) / 2

            # Cover constraint: Ensure all elements in U are covered
            H_cover = 0
            for elem in self.universe:
                cover_count = sum(x[i] for i, subset in enumerate(self.subsets) if elem in subset)
                H_cover += (1 - cover_count) ** 2

            # Objective term: Minimize the number of subsets chosen
            H_objective = np.sum(x)

            # Combine terms
            A, B = 10.0, 1.0  # Weights for constraints and objective
            return A * H_cover + B * H_objective

        return hamiltonian
                
universe = [1, 2, 3, 4]  # Universe U = {1, 2, 3, 4}
subsets = [[1, 2], [2, 3], [3, 4], [1, 4]]  # Subsets S_1 = {1, 2}, etc.

set_cover_problem = SetCover(universe, subsets)

sim = Simulator(len(subsets), set_cover_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([1, -1, 1, -1], 2.0)
Suggested optimal t: 1.0
Quantum Result: ([1, -1, 1, -1], 2.0)
Qiskit Result: ([-1, 1, -1, 1], 2.0)


In [39]:
class Knapsack:
    def __init__(self, values, weights, capacity):
        """
        Initialize the Knapsack problem.
        :param values: List of values for the items.
        :param weights: List of weights for the items.
        :param capacity: Maximum weight capacity.
        """
        self.values = values
        self.weights = weights
        self.capacity = capacity

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Knapsack problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_i = (s_i + 1) / 2)
            x = (np.array(spins) + 1) / 2

            # Weight constraint
            total_weight = np.dot(self.weights, x)
            H_constraint = (total_weight - self.capacity) ** 2

            # Objective term: Maximize value
            H_objective = -np.dot(self.values, x)

            # Combine terms
            A, B = 10.0, 1.0  # Weights for constraints and objective
            return A * H_constraint + B * H_objective

        return hamiltonian
        
values = [10, 40, 30, 50]  # Values of items
weights = [5, 8, 7, 6]  # Weights of items
capacity = 15  # Maximum weight capacity

knapsack_problem = Knapsack(values, weights, capacity)

sim = Simulator(len(values), knapsack_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, 1, -1, 1], -80.0)
Suggested optimal t: 0.1
Quantum Result: ([-1, 1, -1, 1], -80.0)
Qiskit Result: ([-1, 1, -1, 1], -80.0)


In [40]:
class GraphColoring:
    def __init__(self, vertices, edges, num_colors):
        """
        Initialize the Graph Coloring problem.
        :param vertices: List of vertices (e.g., [0, 1, 2, 3]).
        :param edges: List of edges (e.g., [(0, 1), (1, 2), (2, 3), (3, 0)]).
        :param num_colors: Number of available colors.
        """
        self.vertices = vertices
        self.edges = edges
        self.num_colors = num_colors

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Graph Coloring problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_i = (s_i + 1) / 2)
            x = (np.array(spins) + 1) / 2
            x = x.reshape((len(self.vertices), self.num_colors))  # Reshape for vertices and colors

            # Assignment constraint: Each vertex has exactly one color
            H_assignment = 0
            for v in self.vertices:
                H_assignment += (1 - np.sum(x[v, :])) ** 2

            # Coloring constraint: No two adjacent vertices share the same color
            H_coloring = 0
            for (u, v) in self.edges:
                for c in range(self.num_colors):
                    H_coloring += x[u, c] * x[v, c]

            # Combine terms
            A = 10.0  # Weight for constraints
            return A * H_assignment + A * H_coloring

        return hamiltonian

vertices = [0, 1, 2, 3]  # Vertices of the graph
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]  # Edges of the graph
num_colors = 3  # Number of colors

graph_coloring_problem = GraphColoring(vertices, edges, num_colors)

sim = Simulator(len(vertices) * num_colors, graph_coloring_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, 1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1], 0.0)
Suggested optimal t: 0.1
Quantum Result: ([-1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, 1], 0.0)
Qiskit Result: ([1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1], 0.0)


In [41]:
class CliqueCover:
    def __init__(self, vertices, edges, num_cliques):
        """
        Initialize the Clique Cover problem.
        :param vertices: List of vertices (e.g., [0, 1, 2, 3]).
        :param edges: List of edges (e.g., [(0, 1), (1, 2), (2, 3), (3, 0)]).
        :param num_cliques: Number of available cliques.
        """
        self.vertices = vertices
        self.edges = edges
        self.num_cliques = num_cliques

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Clique Cover problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables (x_i = (s_i + 1) / 2)
            x = (np.array(spins) + 1) / 2
            x = x.reshape((len(self.vertices), self.num_cliques))  # Reshape for vertices and cliques

            # Assignment constraint: Each vertex is assigned to exactly one clique
            H_assignment = 0
            for v in self.vertices:
                H_assignment += (1 - np.sum(x[v, :])) ** 2

            # Clique validation constraint
            H_clique = 0
            for i in range(self.num_cliques):
                # Maximum possible edges in the clique
                max_edges = 0.5 * (-1 + np.sum(x[:, i])) * np.sum(x[:, i])
                # Actual edges in the clique
                actual_edges = sum(x[u, i] * x[v, i] for (u, v) in self.edges)
                H_clique += max_edges - actual_edges

            # Combine terms
            A, B = 10.0, 1.0  # Weights for constraints and clique validation
            return A * H_assignment + B * H_clique

        return hamiltonian
            
vertices = [0, 1, 2, 3]  # Vertices of the graph
edges = [(0, 1), (1, 2), (2, 3)]  # Edges of the graph
num_cliques = 2  # Number of cliques

clique_cover_problem = CliqueCover(vertices, edges, num_cliques)

sim = Simulator(len(vertices) * num_cliques, clique_cover_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, 1, -1, 1, 1, -1, 1, -1], 0.0)
Suggested optimal t: 1.0
Quantum Result: ([1, -1, 1, -1, -1, 1, -1, 1], 0.0)
Qiskit Result: ([1, -1, 1, -1, -1, 1, -1, 1], 0.0)


In [42]:
class JobSequencing:
    def __init__(self, num_jobs, num_clusters, job_lengths):
        """
        Initialize the Job Sequencing problem.
        :param num_jobs: Number of jobs (N).
        :param num_clusters: Number of clusters (m).
        :param job_lengths: List of job lengths (L_i).
        """
        self.num_jobs = num_jobs
        self.num_clusters = num_clusters
        self.job_lengths = job_lengths

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Job Sequencing problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables
            x = (np.array(spins) + 1) / 2
            x = x.reshape((self.num_jobs, self.num_clusters))  # Reshape for jobs and clusters

            # Constraint Hamiltonian
            H_constraint = 0
            for i in range(self.num_jobs):
                H_constraint += (1 - np.sum(x[i, :])) ** 2

            # Length difference and cluster assignment constraints
            H_length = 0
            for alpha in range(self.num_clusters):
                M_alpha = np.sum([
                    self.job_lengths[i] * x[i, alpha] for i in range(self.num_jobs)
                ])
                M1 = np.sum([
                    self.job_lengths[i] * x[i, 0] for i in range(self.num_jobs)
                ])
                H_length += (M_alpha - M1) ** 2

            # Combine constraints
            A = 10.0
            H_A = A * (H_constraint + H_length)

            # Objective Hamiltonian
            H_B = np.sum([
                self.job_lengths[i] * x[i, 0] for i in range(self.num_jobs)
            ])
            B = 1.0

            return H_A + B * H_B

        return hamiltonian

num_jobs = 4
num_clusters = 2
job_lengths = [3, 2, 7, 5]

job_sequencing_problem = JobSequencing(num_jobs, num_clusters, job_lengths)

sim = Simulator(num_jobs * num_clusters, job_sequencing_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, 1, -1, 1, -1, -1, 1, -1], 15.0)
Suggested optimal t: 1.0
Quantum Result: ([-1, 1, -1, 1, -1, -1, 1, -1], 15.0)
Qiskit Result: ([-1, 1, -1, 1, -1, -1, 1, -1], 15.0)


In [ ]:
# Incorrect result let's circle back here, quantum result varies.
class HamiltonianCycles:
    def __init__(self, vertices, edges):
        """
        Initialize the Hamiltonian Cycles problem.
        :param vertices: List of vertices (e.g., [0, 1, 2, 3]).
        :param edges: List of edges (e.g., [(0, 1), (1, 2), (2, 3), (3, 0)]).
        """
        self.vertices = vertices
        self.edges = edges
        self.num_vertices = len(vertices)

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Hamiltonian Cycles problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables
            x = (np.array(spins) + 1) / 2
            x = x.reshape((self.num_vertices, self.num_vertices))  # Reshape for vertices and positions

            # Vertex assignment constraint
            H_vertex = 0
            for v in range(self.num_vertices):
                H_vertex += (1 - np.sum(x[v, :])) ** 2

            # Position assignment constraint
            H_position = 0
            for j in range(self.num_vertices):
                H_position += (1 - np.sum(x[:, j])) ** 2

            # Edge validation constraint
            H_edges = 0
            edge_set = set(self.edges)
            for u in range(self.num_vertices):
                for v in range(self.num_vertices):
                    if (u, v) not in edge_set and u != v:
                        for j in range(self.num_vertices - 1):  # For paths
                            H_edges += x[u, j] * x[v, j + 1]

            # Combine terms
            A = 10.0  # Weight for constraints
            return A * (H_vertex + H_position + H_edges)

        return hamiltonian
        
vertices = [0, 1, 2, 3]
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]

hamiltonian_problem = HamiltonianCycles(vertices, edges)

sim = Simulator(len(vertices) ** 2, hamiltonian_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1], 0.0)
Suggested optimal t: 0.1


In [78]:
# Incorrect result let's circle back here, quantum result varies.
class TravelingSalesman:
    def __init__(self, vertices, edges, weights):
        """
        Initialize the Traveling Salesman problem.
        :param vertices: List of vertices.
        :param edges: List of edges [(u, v), ...].
        :param weights: Dictionary with edge weights {(u, v): weight, ...}.
        """
        self.vertices = vertices
        self.edges = edges
        self.weights = weights
        self.num_vertices = len(vertices)

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Traveling Salesman problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables
            x = (np.array(spins) + 1) / 2
            x = x.reshape((self.num_vertices, self.num_vertices))  # Reshape for vertices and positions

            # Hamiltonian cycle constraints
            H_vertex = 0
            for v in range(self.num_vertices):
                H_vertex += (1 - np.sum(x[v, :])) ** 2

            H_position = 0
            for j in range(self.num_vertices):
                H_position += (1 - np.sum(x[:, j])) ** 2

            H_edges = 0
            edge_set = set(self.edges)
            for u, v in edge_set:
                for j in range(self.num_vertices - 1):
                    H_edges += x[u, j] * x[v, j + 1]

            # Objective function: Minimize weights
            H_weight = 0
            for (u, v), weight in self.weights.items():
                for j in range(self.num_vertices - 1):
                    H_weight += weight * x[u, j] * x[v, j + 1]

            # Combine terms
            A = 10.0  # Weight for constraints
            B = 1.0   # Weight for objective
            return A * (H_vertex + H_position + H_edges) + B * H_weight

        return hamiltonian
        
vertices = [0, 1, 2, 3]
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
weights = {(0, 1): 1, (1, 2): 2, (2, 3): 3, (3, 0): 4}
num_vertices = len(vertices)
num_variables = num_vertices ** 2  # Each vertex-position pair is a variable

tsp_problem = TravelingSalesman(vertices, edges, weights)

sim = Simulator(num_variables, tsp_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1], 0.0)
Quantum Result: ([-1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1], 20.0)


In [83]:
class MinimalSpanningTree:
    def __init__(self, vertices, edges, weights, max_degree):
        """
        Initialize the Minimal Spanning Tree problem.
        :param vertices: List of vertices.
        :param edges: List of edges [(u, v), ...].
        :param weights: Dictionary with edge weights {(u, v): weight, ...}.
        :param max_degree: Maximum degree constraint (Delta).
        """
        self.vertices = vertices
        self.edges = edges
        self.weights = weights
        self.max_degree = max_degree
        self.num_vertices = len(vertices)
        self.num_edges = len(edges)

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Minimal Spanning Tree problem."""
        def hamiltonian(spins):
            # Convert spins to binary variables
            y = (np.array(spins[:self.num_edges]) + 1) / 2  # Edge variables
            x = (np.array(spins[self.num_edges:]) + 1) / 2  # Vertex variables

            # Constraint Hamiltonian
            H_vertex_depth = 0
            for v in range(self.num_vertices):
                H_vertex_depth += (1 - np.sum(x[v])) ** 2

            H_edge_depth = 0
            for e, (u, v) in enumerate(self.edges):
                H_edge_depth += (y[e] - np.sum(x[u] * x[v])) ** 2

            H_degree = 0
            for v in range(self.num_vertices):
                H_degree += (np.sum(x[v]) - self.max_degree) ** 2

            H_A = H_vertex_depth + H_edge_depth + H_degree

            # Objective Hamiltonian
            H_B = np.sum([
                self.weights[(u, v)] * y[self.edges.index((u, v))]
                for (u, v) in self.edges
            ])

            A = 10.0  # Weight for constraints
            B = 1.0   # Weight for objective
            return A * H_A + B * H_B

        return hamiltonian
    
vertices = [0, 1, 2, 3]
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
weights = {(0, 1): 1, (1, 2): 2, (2, 3): 3, (3, 0): 4}
max_degree = 2
num_variables = len(edges) + len(vertices)

mst_problem = MinimalSpanningTree(vertices, edges, weights, max_degree)

sim = Simulator(num_variables, mst_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([1, 1, 1, 1, 1, 1, 1, 1], 50.0)
Quantum Result: ([1, 1, 1, 1, 1, 1, 1, 1], 50.0)


In [90]:
# Let's check this one again, quantum gives different results
class SteinerTree:
    def __init__(self, vertices, edges, weights, required_vertices):
        """
        Initialize the Steiner Tree problem.
        :param vertices: List of vertices.
        :param edges: List of edges [(u, v), ...].
        :param weights: Dictionary with edge weights {(u, v): weight, ...}.
        :param required_vertices: Subset of required vertices (U).
        """
        self.vertices = vertices
        self.edges = edges
        self.weights = weights
        self.required_vertices = set(required_vertices)
        self.num_vertices = len(vertices)
        self.num_edges = len(edges)

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Steiner Tree problem."""
        def hamiltonian(spins):
            # Split spins into y_v (vertices) and y_uv (edges)
            y_v = (np.array(spins[:self.num_vertices]) + 1) / 2
            y_uv = (np.array(spins[self.num_vertices:]) + 1) / 2

            # Constraint Hamiltonian
            H_A = 0
            for v in self.vertices:
                if v in self.required_vertices:
                    H_A += (1 - y_v[v]) ** 2
                else:
                    H_A += (y_v[v] - np.sum(y_v[v])) ** 2

            for u, v in self.edges:
                edge_idx = self.edges.index((u, v))
                H_A += (y_uv[edge_idx] - (y_v[u] + y_v[v])) ** 2

            # Objective Hamiltonian
            H_B = np.sum([
                self.weights[(u, v)] * y_uv[self.edges.index((u, v))]
                for (u, v) in self.edges
            ])

            A = 10.0  # Weight for constraints
            B = 1.0   # Weight for objective
            return A * H_A + B * H_B

        return hamiltonian

vertices = [0, 1, 2, 3, 4]
edges = [(0, 1), (1, 2), (2, 3), (3, 4), (0, 4), (1, 3)]
weights = {(0, 1): 1, (1, 2): 2, (2, 3): 3, (3, 4): 4, (0, 4): 5, (1, 3): 6}
required_vertices = [0, 2, 4]
num_variables = len(vertices) + len(edges)

steiner_tree = SteinerTree(vertices, edges, weights, required_vertices)

sim = Simulator(num_variables, steiner_tree.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([1, -1, 1, -1, -1, 1, 1, 1, -1, 1, -1], 21.0)
Quantum Result: ([1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -1], 26.0)


In [97]:
class DirectedFeedbackVertexSet:
    def __init__(self, vertices, edges):
        """
        Initialize the Directed Feedback Vertex Set problem.
        :param vertices: List of vertices.
        :param edges: List of directed edges [(u, v), ...].
        """
        self.vertices = vertices
        self.edges = edges
        self.num_vertices = len(vertices)

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Directed Feedback Vertex Set problem."""
        def hamiltonian(spins):
            # Split spins into y_v (feedback set) and x_{v, i} (height variables)
            y = (np.array(spins[:self.num_vertices]) + 1) / 2  # Feedback set variables
            x = (np.array(spins[self.num_vertices:]) + 1) / 2  # Height variables
            x = x.reshape((self.num_vertices, self.num_vertices))  # Reshape into matrix form

            # Constraint Hamiltonian
            H_A = 0
            for v in range(self.num_vertices):
                H_A += (y[v] - np.sum(x[v, :])) ** 2
            for u, v in self.edges:
                H_A += np.sum([x[u, i] * x[v, j] for i in range(self.num_vertices) for j in range(i + 1)])

            # Objective Hamiltonian
            H_B = np.sum(y)

            A = 10.0  # Weight for constraints
            B = 1.0   # Weight for objective
            return A * H_A + B * H_B

        return hamiltonian
    
vertices = [0, 1, 2, 3]
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
num_vertices = len(vertices)
num_variables = num_vertices + num_vertices ** 2

dfvs_problem = DirectedFeedbackVertexSet(vertices, edges)

sim = Simulator(num_variables, dfvs_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], 0.0)
Quantum Result: ([-1, 1, -1, 1, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, -1, 1, 1, -1], 22.0)


In [ ]:
class UndirectedFeedbackVertexSet:
    def __init__(self, vertices, edges):
        """
        Initialize the Undirected Feedback Vertex Set problem.
        :param vertices: List of vertices.
        :param edges: List of undirected edges [(u, v), ...].
        """
        self.vertices = vertices
        self.edges = edges
        self.num_vertices = len(vertices)
        self.num_edges = len(edges)
        self.num_variables = self.num_vertices + self.num_edges * self.num_vertices  # Variables for y_v, x_v,i, and edge variables

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the Undirected Feedback Vertex Set problem."""
        def hamiltonian(spins):
            # Split spins into y_v and x_v,i
            y = (np.array(spins[:self.num_vertices]) + 1) / 2  # Feedback set variables
            x = (np.array(spins[self.num_vertices:]) + 1) / 2  # Tree depth variables
            x = x.reshape((self.num_edges, self.num_vertices))  # Reshape into matrix form for edges and vertices

            # Constraint Hamiltonian
            H_A = 0
            # Constraint 1: Feedback set or tree depth
            for v in range(self.num_vertices):
                H_A += (1 - y[v] - np.sum(x[:, v])) ** 2

            # Constraint 2: Edges in the feedback set or trees
            for e, (u, v) in enumerate(self.edges):
                H_A += (1 - np.sum(x[e, :]) - y[u] - y[v]) ** 2

            # Constraint 3: Proper edge relationships
            for e, (u, v) in enumerate(self.edges):
                H_A += (y[u] - y[v]) ** 2

            # Objective Hamiltonian
            H_B = np.sum(y)

            # Combine terms
            A = 10.0  # Weight for constraints
            B = 1.0   # Weight for objective
            return A * H_A + B * H_B

        return hamiltonian

vertices = [0, 1, 2, 3]
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
num_vertices = len(vertices)
num_edges = len(edges)
num_variables = num_vertices + num_edges * num_vertices  # Variables for y_v, x_v,i, and edge variables

ufvs_problem = UndirectedFeedbackVertexSet(vertices, edges)

sim = Simulator(num_variables, ufvs_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(t=optimal_t)
print("Quantum Result:", quantum_result)

# Qiskit Simulation
quantum_result = sim.qiskit(t=optimal_t)
print("Qiskit Result:", quantum_result)

Classical Result: ([-1, -1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1], 0.0)
Suggested optimal t: 0.5


In [22]:
def test_single_qubit():
    def hamiltonian(spins):
        # Single qubit Hamiltonian: Z
        return spins[0]
    
    sim = Simulator(N=1, H=hamiltonian)
    eigenvalues = sim.compute_eigenvalues()
    assert eigenvalues == [-1, 1], f"Unexpected eigenvalues: {eigenvalues}"
    print("Test 1 passed: Single Qubit")
test_single_qubit()

Test 1 passed: Single Qubit
